In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import openai
import openpyxl
import os

In [3]:

openai.api_key = "my_api_key"


## Data loading and Preprocessing

In [2]:
df= pd.read_csv('input_table.csv')
df.shape

(1000, 11)

In [3]:
df.head()

,Invoice ID,City,Gender,Product line,Unit price,Quantity,Total,Date,Payment,gross income,Rating
0,750-67-8428,Yangon,Female,Health and beauty,74.69,7,548.9715,1/5/2019,Ewallet,26.1415,9.1
1,226-31-3081,Naypyitaw,Female,Electronic accessories,15.28,5,80.2200,3/8/2019,Cash,3.8200,9.6
2,631-41-3108,Yangon,Male,Home and lifestyle,46.33,7,340.5255,3/3/2019,Credit card,16.2155,7.4
3,123-19-1176,Yangon,Male,Health and beauty,58.22,8,489.0480,1/27/2019,Ewallet,23.2880,8.4
4,373-73-7910,Yangon,Male,Sports and travel,86.31,7,634.3785,2/8/2019,Ewallet,30.2085,5.3


In [4]:
df.isna().sum()

Invoice ID      0
City            0
Gender          0
Product line    0
Unit price      0
Quantity        0
Total           0
Date            0
Payment         0
gross income    0
Rating          0
dtype: int64

In [5]:
df.dtypes

Invoice ID       object
City             object
Gender           object
Product line     object
Unit price      float64
Quantity          int64
Total           float64
Date             object
Payment          object
gross income    float64
Rating          float64
dtype: object

In [6]:
df['Date'] = pd.to_datetime(df['Date'])

In [7]:
df.columns

Index(['Invoice ID', 'City', 'Gender', 'Product line', 'Unit price',
       'Quantity', 'Total', 'Date', 'Payment', 'gross income', 'Rating'],
      dtype='object')

In [11]:
#function to convert row into text

def row_to_text(row):
    return f"Invoice ID: {row['Invoice ID']},City:{row['City']},Gender:{row['Gender']},Product line:{row['Product line']},Unit price:{row['Unit price']},Quantity:{row['Quantity']},Total:{row['Total']},Date:{row['Date']},Payment:{row['Payment']},gross income:{row['gross income']},Rating:{row['Rating']}"  

table_text= df.apply(row_to_text, axis=1).tolist()

In [12]:
#embeddings
embed_model= SentenceTransformer('paraphrase-MiniLM-L6-v2')
table_embeddings= embed_model.encode(table_text).astype(np.float32)

In [13]:
embedding_dim= table_embeddings.shape[1]
embedding_dim

384

In [14]:
# row indexes

row_text = df.apply(lambda row: " | ".join(map(str, row)), axis=1).tolist()
row_embeddings = embed_model.encode(row_text).astype(np.float32)
row_index = faiss.IndexFlatL2(embedding_dim)
row_index.add(row_embeddings)
faiss.write_index(row_index, "faiss_row_index.idx")


In [15]:
# column indices

column_text= [f"Column: {col}" for col in df.columns]
column_embeddings = embed_model.encode(column_text).astype(np.float32)

column_index = faiss.IndexFlatL2(embedding_dim)
column_index.add(column_embeddings)
faiss.write_index(column_index,"faiss_column_index.idx")

In [16]:
print("Row Index Size:", row_index.ntotal)
print("Column Index Size:", column_index.ntotal)


Row Index Size: 1000
Column Index Size: 11


### RAG

In [17]:
#data retrieval based on cosine similarity

def retrieve_relevant_data(query, table_embeddings, column_embeddings, row_index, column_index, threshold=0.5, max_results=10):
    query_embedding = embed_model.encode([query]).astype(np.float32)

    query_embedding = query_embedding / np.linalg.norm(query_embedding)
    table_embeddings = table_embeddings / np.linalg.norm(table_embeddings, axis=1, keepdims=True)
    column_embeddings = column_embeddings / np.linalg.norm(column_embeddings, axis=1, keepdims=True)
 
    _, row_indices = row_index.search(query_embedding, max_results)
    _, column_indices = column_index.search(query_embedding, max_results)
    
    row_similarities = row_indices[0]
    column_similarities = column_indices[0]

    relevant_row_indices = [idx for idx, sim in zip(row_indices[0], row_similarities) if sim >= threshold]
    relevant_column_indices = [idx for idx, sim in zip(column_indices[0], column_similarities) if sim >= threshold]

    relevant_rows = df.iloc[relevant_row_indices].values.tolist() if relevant_row_indices else []
    relevant_columns = [df.columns[idx] for idx in relevant_column_indices] if relevant_column_indices else []

    return relevant_rows, relevant_columns, relevant_row_indices, relevant_column_indices


In [18]:
## checking if it works

query = "What product line is in the latest entry?"
threshold = 0.7  
max_results = 10  

relevant_rows, relevant_columns, relevant_row_indices, relevant_column_indices = retrieve_relevant_data(query, table_embeddings, column_embeddings, row_index, column_index, threshold=0.5, max_results=10)

print("Relevant Rows :")
for row in relevant_rows:
    print(row)

print("\nRelevant colss:")
for col in relevant_columns:
    print(col)

print("\nRelevant row index:")
for row in relevant_row_indices:
    print(row)

print("\nRelevant cols index:")
for col in relevant_column_indices:
    print(col)

Relevant Rows :
['232-16-2483', 'Naypyitaw', 'Female', 'Sports and travel', 68.12, 1, 71.526, Timestamp('2019-01-07 00:00:00'), 'Ewallet', 3.406, 6.8]
['373-73-7910', 'Yangon', 'Male', 'Sports and travel', 86.31, 7, 634.3785, Timestamp('2019-02-08 00:00:00'), 'Ewallet', 30.2085, 5.3]
['236-86-3015', 'Naypyitaw', 'Male', 'Home and lifestyle', 13.98, 1, 14.679, Timestamp('2019-02-04 00:00:00'), 'Ewallet', 0.699, 9.8]
['408-26-9866', 'Naypyitaw', 'Female', 'Sports and travel', 73.98, 7, 543.753, Timestamp('2019-03-02 00:00:00'), 'Ewallet', 25.893, 4.1]
['731-81-9469', 'Naypyitaw', 'Female', 'Sports and travel', 89.8, 10, 942.9, Timestamp('2019-01-23 00:00:00'), 'Credit card', 44.9, 5.4]
['645-78-8093', 'Yangon', 'Female', 'Sports and travel', 93.14, 2, 195.594, Timestamp('2019-01-20 00:00:00'), 'Ewallet', 9.314, 4.1]
['803-83-5989', 'Naypyitaw', 'Male', 'Home and lifestyle', 55.73, 6, 351.099, Timestamp('2019-02-24 00:00:00'), 'Ewallet', 16.719, 7.0]
['873-51-0671', 'Yangon', 'Female', 'S

we are able to sucessfully retrieve desired columns and rows as well as their indexes

In [36]:
# function to generate data given to llms

def imp_row_col(query, threshold=0.5, max_results=20):
    relevant_rows, relevant_columns, relevant_row_indices, relevant_column_indices = retrieve_relevant_data(query, table_embeddings, column_embeddings, row_index, column_index, threshold=0.5, max_results=10)
    
    context = "" 

    context += "Relevant Rows:\n"
    for row in relevant_rows:
        context += f"{row}`\n"

    context += "\nRelevant Columns:\n"
    for col in relevant_columns:
        context += f"{col}\n"
    
    return context


In [21]:
# exampple:

value= imp_row_col("What is the average unit price?",0.5,10)
value

"Relevant Rows:\n['232-16-2483', 'Naypyitaw', 'Female', 'Sports and travel', 68.12, 1, 71.526, Timestamp('2019-01-07 00:00:00'), 'Ewallet', 3.406, 6.8]\n['236-86-3015', 'Naypyitaw', 'Male', 'Home and lifestyle', 13.98, 1, 14.679, Timestamp('2019-02-04 00:00:00'), 'Ewallet', 0.699, 9.8]\n['803-83-5989', 'Naypyitaw', 'Male', 'Home and lifestyle', 55.73, 6, 351.099, Timestamp('2019-02-24 00:00:00'), 'Ewallet', 16.719, 7.0]\n['373-73-7910', 'Yangon', 'Male', 'Sports and travel', 86.31, 7, 634.3785, Timestamp('2019-02-08 00:00:00'), 'Ewallet', 30.2085, 5.3]\n['645-78-8093', 'Yangon', 'Female', 'Sports and travel', 93.14, 2, 195.594, Timestamp('2019-01-20 00:00:00'), 'Ewallet', 9.314, 4.1]\n['848-95-6252', 'Naypyitaw', 'Female', 'Home and lifestyle', 86.27, 1, 90.5835, Timestamp('2019-02-20 00:00:00'), 'Ewallet', 4.3135, 7.0]\n['534-53-3526', 'Yangon', 'Female', 'Sports and travel', 94.76, 4, 397.992, Timestamp('2019-02-11 00:00:00'), 'Ewallet', 18.952, 7.8]\n['408-26-9866', 'Naypyitaw', 'Fe

## Fetching answer with LLM

In [47]:
## function to generate answer


def generate_ans(query, context):
    prompt = f"""
    You are an AI assistant that processes tabular data and answers questions based on it. try your best to answers in word rather than sentence

    Example Questions and Answers:
    
    question: What product line is in the latest entry?
    answer: Fashion accessories
    
    question: On what date did the first transaction occur?
    answer: 1/1/2019
    
    question: How many transactions involved Male customers and a rating of 9.1?
    answer: 5
    
    Now, process the given tabular data context and answer the following question.

    Context:
    {context}

    question: {query}
    answer:
    """

    response = openai.chat.completions.create(
        model="gpt-4o-mini",  
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content.strip()


In [26]:
## example

answer = generate_ans("What product line is in the latest entry?", value)
print("Answer:", answer)


Answer: Home and lifestyle


## trying to fill qa file

In [48]:
input= pd.read_excel("QA_dataset_share.xlsx")
input.columns

Index(['question', 'row index', 'column index', 'answer', 'filtered row index',
       'filtered column index', 'generated response'],
      dtype='object')

In [49]:
def fill_index(df, threshold=0.5, max_results=10):
    for i, query in enumerate(df['question']):
        relevant_rows, relevant_columns, relevant_row_index, relevant_column_index= retrieve_relevant_data(query,table_embeddings, column_embeddings, row_index, column_index, threshold, max_results)
        df.at[i, 'filtered row index'] = ', '.join(map(str, relevant_row_index))
        df.at[i, 'filtered column index'] = ', '.join(map(str, relevant_column_index))


fill_index(input,0.7,15 )

C:\Users\varsh\AppData\Local\Temp\ipykernel_2384\1810989406.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '35, 4, 402, 569, 140, 235, 61, 92, 781, 535, 860, 163, 982, 470, 597' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, 'filtered row index'] = ', '.join(map(str, relevant_row_index))
C:\Users\varsh\AppData\Local\Temp\ipykernel_2384\1810989406.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3, 6, 1, 2, 7, 10, 4, 5, 9, 8' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, 'filtered column index'] = ', '.join(map(str, relevant_column_index))


In [35]:
# input.to_csv("filled file.csv")

In [50]:
# Function to fill answers in the DataFrame
def fill_answer(df, threshold=0.5, max_results=10):
    for i, query in df['question'].items():  
        context_value = imp_row_col(query, threshold, max_results)  
        answer = generate_ans(query, context_value)  
        df.at[i, 'generated_response'] = answer  


In [51]:
fill_answer(input, 0.5,15)

In [52]:
input['generated_response']

0     Home and lifestyle
1               1/1/2019
2       answer: 3/2/2019
3                    9.8
4                      0
             ...        
65         answer: 13.98
66                     1
67             answer: 0
68                     2
69                     0
Name: generated_response, Length: 70, dtype: object

In [53]:
input.to_excel("filled qa file.xlsx")